In [1]:
import re
import fitz  
import pathlib as pl
import os
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import json
import os
import nltk

folder = r"C:\Users\Ilias\Desktop\WB Internship\PCP\cv"
words = set(nltk.corpus.words.words())

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2  
    return df_1

def get_files(folder):
    """
    get names of all pdf files in the folder
    """
    os.chdir(folder)
    files = os.listdir()
    files = [x for x in files if x.endswith(".pdf")]
    return files 

def compute_years_edu(edu):
    years = 0
    if 'b.a.' in edu:
        years = years + 4
    if 'b.b.a.' in edu:
        years = years + 4 
    if 'b.com.' in edu:
        years = years +3 
    if 'b.e.' in edu:
        years = years + 4 
    if 'b.s.' in edu:
        years = years + 4
    if 'b.sc.' in edu:
        years = years + 4 
    if 'l.l.b.' in edu:
        years = years +3 
    if 'l.l.m.' in edu:
        years = years + 2 
    if 'm.a.' in edu:
        years = years +2 
    if 'm.b.a.' in edu:
        years = years + 2 
    if 'm.com.' in edu:
        years = years + 2
    if 'm.p.a.' in edu:
        years = years + 2 
    if 'm.phil.' in edu:
        years = years + 2  
    if 'm.s.' in edu:
        years = years +2 
    if 'm.sc.' in edu:
        years = years + 2 
    if 'mbbs' in edu:
        years = years + 5
    if 'p.g.d' in edu:
        years = years + 4 
    return years

def create_lang_col(df):
    df['worker_eng'] = df['languages'].apply(lambda x: 1 if 'english' in x else 0)
    df['worker_urdu'] = df['languages'].apply(lambda x: 1 if 'urdu' in x else 0)
    df['worker_kashmiri'] = df['languages'].apply(lambda x: 1 if 'kashmiri' in x else 0)
    df['worker_pashto'] = df['languages'].apply(lambda x: 1 if 'pashto' in x else 0)
    df['worker_punjabi'] = df['languages'].apply(lambda x: 1 if 'punjabi' in x else 0)
    df['worker_sindhi'] = df['languages'].apply(lambda x: 1 if 'sindhi' in x else 0)


def extract_training_table(text_training):
    if len(text_training[2:]) > 0:        
        df_tr = pd.DataFrame({'training':text_training[1:]})
        df_tr = df_tr.training.str.split(pat='\n', expand = True)
        df_tr.columns = df_tr.iloc[0]
        df_tr = df_tr.add_prefix('worker_training_')
        df_tr = df_tr.iloc[1:,0:5]
        df_tr.columns= df_tr.columns.str.lower()             
    else: 
        df_tr = pd.DataFrame({'worker_training_institute': [''], 'worker_training_country': [''], 
                              'worker_training_from' :[''], 'worker_training_to':[''], 'worker_training_course':['']})
    return df_tr

 
def make_dfs(text_demo, text_qualification, text_training, text_post):
# extract demographics information
    df_demo = pd.DataFrame([text_demo])
    df_demo.rename(columns = {0: 'worker_name', 1: 'worker_occupational', 2: 'worker_dob', 3:'date_of_joining_govt_service',
                        4: 'length_of_service', 5: 'worker_current_position', 6: 'date_of_present_appointment', 
                        7: 'worker_organization', 8:'seniority_no'}, inplace = True)

    # remove text redundancy
    df_demo[['worker_name', 'worker_occupational', 'worker_dob', 'date_of_joining_govt_service', 
        'date_of_present_appointment', 'seniority_no']] = df_demo[['worker_name', 'worker_occupational', 'worker_dob', 'date_of_joining_govt_service', 
        'date_of_present_appointment', 'seniority_no']].apply(lambda x: x.str.extract(r'\:(.*)', expand=False))

    df_demo['worker_prov_dist'] = df_demo.iloc[:,0].str.extract(r'\[(.*)\]', expand=False)
    df_demo['worker_name'] = df_demo['worker_name'].apply(lambda x: x.split(sep='[')[0])
    df_demo['worker_birth_district'] = df_demo['worker_prov_dist'].str.extract(r'\((.*)\)', expand=False)
    df_demo['worker_birht_province'] = df_demo['worker_prov_dist'].str.split(pat='(', expand = True)[0]
    df_demo[['worker_dob', 'worker_superannuation']] = df_demo['worker_dob'].str.split('DATE OF SUPERANNUATION:', expand = True)
    df_demo.drop(columns='worker_prov_dist', inplace = True)
    
    # extract qualification information
    df_qual = pd.DataFrame({'qualification':text_qualification[2:]})
    df_qual = df_qual.qualification.str.split(pat='\n', expand = True)
    df_qual.rename(columns = {0: 'academic_professional', 1: 'languages'}, inplace = True)
    df_qual = df_qual[['academic_professional','languages']]
    
    # extract training information
    df_tr = extract_training_table(text_training)
    
    # extract posting information
    df_post = pd.DataFrame({'posting':text_post[2:]})
    df_post = df_post.posting.str.split(pat='\n', expand = True)
    df_post.rename(columns = {0: 'worker_post', 1: 'BS', 2: 'post_from', 3:'post_to',
                        4: 'worker_post_organization'}, inplace = True)


    return df_demo, df_qual, df_tr, df_post

def clean_col(df):
    """
    strip from the dateframe whitespace, commas, slashes and new lines
    """
    df = df.apply(lambda x: x.str.strip(', ').str.lower())
    df = df.apply(lambda x: x.str.strip('\\'))
    df = df.apply(lambda x: x.str.strip('\n'))
    return df



C:\Users\Ilias\miniconda3\envs\pcp\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
names = get_files(folder)
df_post_dict = pd.read_excel(r'C:\Users\Ilias\Desktop\WB Internship\PCP\dictionary\position_grouped_dictionary.xlsx')
df_demo_all = []
df_post_all = []
df_tr_all = []
for i in names: 
    with fitz.open(i) as doc:
        lines = []
        for page in doc:
            page_text_block = page.get_text('blocks')
            for i in page_text_block:
                lines.append(i[4])
        
            idxs_qual = [i for i, item in enumerate(lines) if item.startswith('QUALIFICATION')]
            idxs_train = [i for i, item in enumerate(lines) if item.startswith('TRAINING DETAILS')]
            idxs_post = [i for i, item in enumerate(lines) if item.startswith('POSTINGS')]
            text_demo = lines[1:idxs_qual[0]]
            text_qualification = lines[idxs_qual[0]:idxs_train[0]]
            text_training = lines[idxs_train[0]:idxs_post[0]] 
            text_post = lines[idxs_post[0]:]
            if len(text_demo) == 8:
                text_demo.insert(7,'')
            
            # creating tables from individual lists     
            df_demo, df_qual, df_tr, df_post = make_dfs(text_demo, text_qualification, text_training, text_post)
            
            # combining demographics data with qualification data
            df_demo_comb = pd.concat([df_demo, df_qual], axis = 1)
            df_demo_all.append(df_demo_comb)
            
            # duplicating rows and then combining with training data
            df_demo_comb_dupl_tr = pd.DataFrame(np.repeat(df_demo_comb.values, df_tr.shape[0], axis=0), 
                                 columns=df_demo_comb.columns)
            
            df_demo_comb_dupl_tr.reset_index(drop=True, inplace=True)
            df_tr.reset_index(drop=True, inplace=True)
            df_tr_comb = pd.concat([df_demo_comb_dupl_tr['worker_name'], df_tr], axis = 1)            
            df_tr_all.append(df_tr_comb)
            
            #duplicating rows and then combining with posting data
            df_demo_comb_dupl_post = pd.DataFrame(np.repeat(df_demo_comb.values, df_post.shape[0], axis=0), 
                     columns=df_demo_comb.columns)
            df_post_comb = pd.concat([df_demo_comb_dupl_post['worker_name'], df_post], axis = 1)
            df_post_all.append(df_post_comb)

            
df_demo_all = pd.concat(df_demo_all)  
df_demo_all['length_of_service'] = df_demo_all['length_of_service'].apply(lambda x: x.split('HPSB')[0]) 
df_demo_all = clean_col(df_demo_all)
df_demo_all['length_of_service']= df_demo_all['length_of_service'].apply(lambda x: x.split('length of service in ')[1])
df_demo_all['worker_years_education'] = df_demo_all['academic_professional'].apply(lambda x: compute_years_edu(x))
create_lang_col(df_demo_all)
df_demo_all.drop_duplicates(inplace=True)

df_demo_all['worker_current_bs'] = df_demo_all['worker_current_position'].apply(lambda x: re.findall('\((.*?)\)', x )[-1])
df_demo_all['worker_current_position'] = df_demo_all['worker_current_position'].apply(lambda x:re.sub(r'\((.*?)\)','', x))
df_demo_all['worker_current_position'] = df_demo_all['worker_current_position'].apply(lambda x: x.strip(' \n')) 

df_demo_merged = pd.merge(df_demo_all, df_post_dict, how= 'left', left_on = 'worker_current_position', right_on = 'cv_post')
df_demo_merged.drop(columns = 'cv_post', inplace = True)
idx = df_demo_merged.columns.get_loc('worker_current_position')
df_demo_merged.insert(loc=idx+1, column='worker_current_position_grouped', value=df_demo_merged.grouped_post)
df_demo_merged.drop(columns = ['grouped_post'], inplace = True)



df_tr_all = pd.concat(df_tr_all)
df_tr_all.drop_duplicates(inplace = True)
df_tr_all = clean_col(df_tr_all)


df_post_all = pd.concat(df_post_all)
df_post_all.drop_duplicates(inplace= True)
df_post_all = clean_col(df_post_all)

# remove non-datetime values from the column post_from and post_to
non_datetime_vls = df_post_all.post_to.str.contains(pat=r'[a-zA-Z]+')
df_post_all.loc[non_datetime_vls,'worker_post_organization'] = df_post_all.loc[non_datetime_vls,'post_to'] + ';' + \
                                        df_post_all.loc[non_datetime_vls,'worker_post_organization']
df_post_all.loc[non_datetime_vls,'post_to'] = ''
non_datetime_vls_from = df_post_all.post_from.str.contains(pat=r'[a-zA-Z]+')
df_post_all.loc[non_datetime_vls_from,'worker_post_organization'] = df_post_all.loc[non_datetime_vls_from,'post_from'] + ';' + \
                                        df_post_all.loc[non_datetime_vls_from,'worker_post_organization']
df_post_all.loc[non_datetime_vls_from,'post_from'] = ''



# add column, that contains grouped positions from the column post
df_post_dict['grouped_post'] = df_post_dict['grouped_post'].str.strip()
df_post_merged = pd.merge(df_post_all, df_post_dict, how= 'left', left_on = 'worker_post', right_on = 'cv_post')
df_post_merged.drop(columns = 'cv_post', inplace = True)
idx = df_post_merged.columns.get_loc('worker_post')
df_post_merged.insert(loc=idx+1, column='worker_post_grouped', value=df_post_merged.grouped_post)
df_post_merged.drop(columns = ['grouped_post',5], inplace = True)


with pd.ExcelWriter('output.xlsx') as writer:  
    df_demo_merged.to_excel(writer, sheet_name = 'Demographics', index = False)
    df_post_merged.to_excel(writer, sheet_name='Postings', index = False)
    df_tr_all.to_excel(writer, sheet_name='Trainings', index = False)